# Bayesian Phylogenetics in Linguistics explained

This is a Jupyter notebook that shows you a very simplified example of Bayesian phylogenetics in linguistics, step by step, from the initial data to a resulting summary tree.

## Program libraries
The code here is written in Python. Python comes with a lot of libraries to deal with numerical processes, trees, plotting, and so on; I load them here.

In [243]:
%load_ext autoreload
%autoreload 2
import numpy, ete3, newick, pandas as pd
DF = pd.DataFrame
from matplotlib import pyplot as plt
from helpers import roll_die
import model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Language data

Now let us do some Bayesian phylogenetics, starting from a very simple and constructed example. Let us say we have some data like this.

I use lexical data in my example because it is the data type best understood for computational models. The methodology is not specific to it, as I will explain below; Having a different type of data, and corresponding models of language change, will mean that the rest of the methodology can be applied equivalently.


In [280]:
raw_data = DF({
  "Kalang": ['tuˈmun', 'pak', 'ˈrata'],
  "Ta'e": ['ˈana', 'ˈʋula', 'ˈrata'],
  "Sunggama": ['anˈa', 'wuˈla', 'deˈna'],
  "Nda'o": ['ˈana', 'ˈwuɹa', 'ˈn͡dena']},
  index=["child", "moon", "flat"])
data

   Kl  Nd  Sg  Ta
0   1   2   2   2
1   1   2   2   2
2   1   2   2   1

I will not explain automatic cognate coding in this notebook, so let us for now assume that it can be done and that automatic or manual cognate coding gives us the following cognate table corresponding to the one above.


In [245]:
data = DF({
  "Kl": [1, 1, 1],
  "Ta": [2, 2, 1],
  "Sg": [2, 2, 2],
  "Nd": [2, 2, 2]})
data

   Kl  Nd  Sg  Ta
0   1   2   2   2
1   1   2   2   2
2   1   2   2   1

## Stochastic processes for creating trees

The basic mathematical theorem underlying this kind of inference explains the data as generated by a stochastic process. How might a stochastic process of language change look like? Let us imagine the following graphic process of how languages might split up. This process is a bit bold and simple, but it is the best compromise connecting the mathematical intuition behind the mathematical structure called the “Yule tree prior” and descriptive population dynamics that I could imagine.

Imagine an island region with difficult navigation between the islands, with one island, and later more, supporting an isolated language community with no contact to any other island.

Once in a generation, each inhabited island sends out a boat to try to cross the turbulent seas and settle a new island. Five out of six boats sink in the process, but one in six boats reaches an uninhabited shore and founds a new village there, following the same tradition.

How could we simulate this process?

First, we need a function that describes the turbulences of the seas. Let us use a roll of a six-sided dice, likes this: Press Ctrl-Enter in the following cell to re-roll the die.


In [246]:
print(roll_die())

⚁


With that, we can model the crossing of the sea by “Roll a die. Iff you roll a ⚀, the crossing is successful and a new village/speech community is founded.” We can now write a function that takes the language tree at a given point in time and propagates it by one generation by rolling the die for each village. For now, I shall use “a” for the ancestral village, “•” for descendants of a village that stayed and “Δ” for successful sailors.


In [247]:
def extend_by_one_generation(tree, success="⚀"):
    for village in tree.get_leaves():
        if roll_die() in success:
            village.add_descendant(newick.Node("{:}Δ".format(village.name), "1"))
            village.add_descendant(newick.Node("{:}•".format(village.name), "1"))
        else:
            village.name += "•"
            village.length = village.length + 1
    return tree

def many_generations_later(tree=None, n_generations=7, success="⚀"):
    if tree is None:
        tree = newick.Node("v", "1")
    for i in range(n_generations):
        extend_by_one_generation(tree, success)
    return tree

In [248]:
tree = many_generations_later(newick.Node("a", "1"))
print(tree.ascii_art())

                      ┌─a••Δ••Δ•
           ┌─a••Δ••───┤
           │          └─a••Δ••••
──a••──────┤
           │          ┌─a••••Δ••
           └─a••••────┤
                      └─a•••••••


In [249]:
print(tree.newick)

((a••Δ••Δ•:2.0,a••Δ••••:2.0)a••Δ••:3.0,(a••••Δ••:3.0,a•••••••:3.0)a••••:2.0)a••:3.0


This now describes a stochastic process of population spread. A different model might be that the sea is much less dangerous and the crossing succeeds in about half the cases, which we model as a dice roll of ⚀⚁⚂.

We still assume that migration is always to a new, uninhabited place, and with no contact back. Both assumptions make the mathematical calculations sufficiently easy that I can both show them here, and use them in the computer models I run.


In [250]:
big_tree = many_generations_later(newick.Node("a", "1"), success="⚀⚁⚂")
print(big_tree.ascii_art())


                                                       ┌─a•ΔΔ•ΔΔΔ
                                            ┌─a•ΔΔ•ΔΔ──┤
                                            │          └─a•ΔΔ•ΔΔ•
                                 ┌─a•ΔΔ•Δ───┤
                                 │          │          ┌─a•ΔΔ•Δ•Δ
                                 │          └─a•ΔΔ•Δ•──┤
                      ┌─a•ΔΔ•────┤                     └─a•ΔΔ•Δ••
                      │          │          ┌─a•ΔΔ•••Δ
                      │          └─a•ΔΔ•••──┤
                      │                     └─a•ΔΔ••••
           ┌─a•Δ──────┤
           │          │                     ┌─a•Δ•Δ•Δ•
           │          │          ┌─a•Δ•Δ•───┤
           │          │          │          │          ┌─a•Δ•Δ••Δ
           │          │          │          └─a•Δ•Δ••──┤
           │          │          │                     └─a•Δ•Δ•••
           │          └─a•Δ•─────┤
           │                     │                     ┌─a•Δ••Δ•Δ
     

If you re-run these notebook cells a few times (Ctrl-Enter), or if you have a good mathematical intuition, you will notice that the second tree ends up usually far bigger than the first tree.

## Bayes' Theorem

Now, how does this help us reconstruct language trees? The example lets me show you how to use a mathematical property of probabilities, known as “Bayes' Theorem” after 18th century scholar Thomas Bayes. The theorem relates conditional probabilities. For our context here, “conditional probability” should be read as “How compatible are two facts”.

For example, a high probability P(A|B) means that the fact A is very compatible with the fact B. A low ‘marginal’ or ‘prior’ probability P(B) can be read as a quantitative way of phrasing ‘I doubt B’.

So let us assume we are really unsure whether the sea is difficult (⚀⚁⚂) or nigh impossible (⚀) to cross, so we take P(⚀⚁⚂)=P(⚀)=0.5

Now we go to the region, and after some research there, we gather the data given above, which convinces us [P(X)=1] of the following two facts.

- There are exactly these four languages spoken in the island region, no more.
- There was a single ancestral village with the tradition described above 7 generations ago.

How compatible is “the sea is difficult to cross” or “the seas is nigh impossible to cross” with this new data?

Bayes' Theorem tells us that

     P(S|D) = P(D|S) * P(S) / P(D)

In this formula, S represents the possible options for the sea, or more generally our model. D is the data, in this case “there are four villages”.
The `P(D|S)` in that formula is the reason we built the computer model above: This conditional probability is not just a measure of compatibility of beliefs, it is also a repeatable experiment like you might think of when you hear the term “probability”: We can run our two models many, many times and count how often we see `D`, that is, how often the tree has exactly 4 leaves.



In [251]:
# P(4 languages | 7 generations, ⚀)
p_one = numpy.mean([len(many_generations_later(n_generations=7, success="⚀").get_leaves())==4 for _ in range(10000)]) * 0.5
p_one

0.06115

In [252]:
# P(4 languages | 7 generations, ⚀⚁⚂)
p_three = numpy.mean([len(many_generations_later(n_generations=7, success="⚀⚁⚂").get_leaves())==4 for _ in range(10000)]) * 0.5
p_three


0.0131

We do not actually have P(D), but we can still calculate the P(S|D) values, the “posterior probabilities”, because we know they are probabilities, so their sum must be one.


In [253]:
p_one, p_three = p_one / (p_one + p_three), p_three / (p_one + p_three)
p_one, p_three

(0.8235690235690235, 0.1764309764309764)


After looking at the data, we see that the ⚀ model is much more convincing than the ⚀⚁⚂ model.
Where we were really unsure before, now we are quite convinced that ⚀ is correct.
We can even put a number on how much better it is: The “Bayes Factor” in favour of ⚀ is


In [254]:
p_one / p_three

4.66793893129771

## Models of language evolution

Now we have seen the basic principle of Bayes' Theorem in action, we can turn to language data.

Just like for the split of speaker communities, we need an explicit stochastic model that describes how languages change over time. Let us take a stochastic version of the basic assumption of glottochronology and assume that words are replaced by new, unrelated words at random, but averaging out to some fixed underlying speed.

This is an obvious simplification, chosen for the illustration purposes here. The models actually used in phylogenetic inference are also always vast simplifications, but hopefully at least somewhat more robust than the one presented here.

A graphical version of this would be that every generation, let us say just before the boat leaves, the whole speaker community comes together, discusses a set of words – for example, “child”, “moon”, and “flat” from our example data above, and then casts lots. Five out of forty lots mean they invent a new word for this concept, which they use from then on.

We can simulate this decision process by taking a deck of cards

In [255]:
all_cards = """🂱 🂲 🂳 🂴 🂵 🂶 🂷 🂸 🂹 🂺
🂡 🂢 🂣 🂤 🂥 🂦 🂧 🂨 🂩 🂪
🃁 🃂 🃃 🃄 🃅 🃆 🃇 🃈 🃉 🃊
🃑 🃒 🃓 🃔 🃕 🃖 🃗 🃘 🃙 🃚""".split()

def draw_card():
    return all_cards[numpy.random.randint(len(all_cards))]


from which we reveal one, and then put it back and shuffle for the next lottery

In [256]:
print(draw_card())

🃔


If the card drawn is one of 🂱, 🂲, 🂳, 🂴, 🂵, the language community invents a new word for that sacred concept, which is not replaced through other means, only possibly sound changes. As a Python function that simulates the evolution of a single concept on a given language tree, this model looks as follows.

In [257]:
def dollo_model_on_tree(tree, new_form="🂱 🂲 🂳 🂴 🂵".split(), existing_forms=None):
    if existing_forms is None:
        existing_forms = set()
    data = {}
    form = max(existing_forms, default=-1) + 1
    existing_forms.add(form)
    for node in tree.walk(mode="preorder"):
        for i in range(int(node.length)):
            if draw_card() in new_form:
                form += 1
                existing_forms.add(form)
        data[node.name] = form
    return data



If we run this model on the big tree from above

In [258]:
print(big_tree.ascii_art())

                                                       ┌─a•ΔΔ•ΔΔΔ
                                            ┌─a•ΔΔ•ΔΔ──┤
                                            │          └─a•ΔΔ•ΔΔ•
                                 ┌─a•ΔΔ•Δ───┤
                                 │          │          ┌─a•ΔΔ•Δ•Δ
                                 │          └─a•ΔΔ•Δ•──┤
                      ┌─a•ΔΔ•────┤                     └─a•ΔΔ•Δ••
                      │          │          ┌─a•ΔΔ•••Δ
                      │          └─a•ΔΔ•••──┤
                      │                     └─a•ΔΔ••••
           ┌─a•Δ──────┤
           │          │                     ┌─a•Δ•Δ•Δ•
           │          │          ┌─a•Δ•Δ•───┤
           │          │          │          │          ┌─a•Δ•Δ••Δ
           │          │          │          └─a•Δ•Δ••──┤
           │          │          │                     └─a•Δ•Δ•••
           │          └─a•Δ•─────┤
           │                     │                     ┌─a•Δ••Δ•Δ
     

we get the following pattern in all the nodes


In [259]:
dollo_model_on_tree(big_tree)

{'a•': 0,
 'a•Δ': 0,
 'a•ΔΔ•': 0,
 'a•ΔΔ•Δ': 0,
 'a•ΔΔ•ΔΔ': 0,
 'a•ΔΔ•ΔΔΔ': 1,
 'a•ΔΔ•ΔΔ•': 1,
 'a•ΔΔ•Δ•': 1,
 'a•ΔΔ•Δ•Δ': 1,
 'a•ΔΔ•Δ••': 1,
 'a•ΔΔ•••': 1,
 'a•ΔΔ•••Δ': 1,
 'a•ΔΔ••••': 1,
 'a•Δ•': 1,
 'a•Δ•Δ•': 1,
 'a•Δ•Δ•Δ•': 2,
 'a•Δ•Δ••': 2,
 'a•Δ•Δ••Δ': 2,
 'a•Δ•Δ•••': 2,
 'a•Δ••': 2,
 'a•Δ••Δ•': 2,
 'a•Δ••Δ•Δ': 2,
 'a•Δ••Δ••': 3,
 'a•Δ•••': 3,
 'a•Δ•••Δ•': 4,
 'a•Δ••••': 4,
 'a•Δ••••Δ': 4,
 'a•Δ•••••': 4,
 'a•••': 5,
 'a•••Δ•': 5,
 'a•••Δ•Δ': 5,
 'a•••Δ•ΔΔ': 5,
 'a•••Δ•Δ•': 5,
 'a•••Δ•••': 5,
 'a••••': 5,
 'a••••Δ•': 5,
 'a••••Δ•Δ': 5,
 'a••••Δ••': 5,
 'a•••••••': 6}

## Monte Carlo sampling

Now we have a stochastic model for the underlying relationships between the languages, and a model for how that three influences the evolution of the lexicon of these languages, we can randomly generate trees and lexicon data on them. When the lexicon data matches our observed data, we note it down in a list. This list will give us an impression of what the actual language history looks like, provided the models are a reasonable approximation of reality.

This strategy of “generate random data and use that in calculations” is called “Monte Carlo sampling”, after the casino in Monaco.

(Side remark: In this case, the models are a perfect representation of reality, because I generated the example data myself using exactly these models ὠ9)


In [260]:
for i, tree in enumerate(model.naive_monte_carlo(tree_generator=many_generations_later, data=data.to_dict('list'))):
    if tree:
      print(i, tree)
    if i > 100:
        break


Wrong number of leaves: (((v•ΔΔ••Δ•:2.0,v•ΔΔ••••:2.0)v•ΔΔ••:3.0,v•Δ•••••:5.0)v•Δ:1,(v•••Δ•••:4.0,v•••••••:4.0)v•••:2.0)v•:2.0
Wrong number of leaves: v•••••••:8.0
Wrong number of leaves: (((v••ΔΔ•Δ•:2.0,v••ΔΔ•••:2.0)v••ΔΔ•:2.0,v••Δ••••:4.0)v••Δ:1,(v•••Δ•••:4.0,(v••••••Δ:1,v•••••••:1)v••••••:3.0)v•••:1)v••:3.0
Wrong number of leaves: (vΔ••••••:7.0,(v••Δ••••:5.0,v•••••••:5.0)v••:2.0)v:1
Wrong number of leaves: v•••••••:8.0
Wrong number of leaves: v•••••••:8.0
Wrong number of leaves: ((((v•ΔΔΔ•Δ•:2.0,v•ΔΔΔ•••:2.0)v•ΔΔΔ•:2.0,(v•ΔΔ••Δ•:2.0,(v•ΔΔ•••Δ:1,v•ΔΔ••••:1)v•ΔΔ•••:1)v•ΔΔ••:2.0)v•ΔΔ:1,v•Δ•••••:5.0)v•Δ:1,(v••Δ••••:5.0,(v••••••Δ:1,v•••••••:1)v••••••:4.0)v••:1)v•:2.0
Wrong number of leaves: v•••••••:8.0
Wrong number of leaves: v•••••••:8.0
Wrong number of leaves: (v•••Δ•••:4.0,v•••••••:4.0)v•••:4.0
Wrong number of leaves: (v••••••Δ:1,v•••••••:1)v••••••:7.0
Not generating the right data: (vΔ••••••:7.0,((v•••ΔΔ••:3.0,v•••Δ•••:3.0)v•••Δ:1,v•••••••:4.0)v•••:3.0)v:1
Wrong number of leaves: (((

## Markov chain Monte Carlo

This looks like a very slow process, because most of the trees have the wrong number of leaves.
If we discard the trees that do not have the right number of leaves, and instead remember the last tree that did, we have a much better chance to find matches.

This “remember one previous thing” is a core property of a Markov chain, so this turns our sampler into something like a Markov chain Monte Carlo sampler. Such an MCMC can employ even more clever shortcuts if more is known analytically about the probabilities of the objects involved. But for the purposes of this illustration, it should be sufficient.


In [262]:
tree_state = None
def random_tree_with_memory(leaves=4):
    global tree_state
    suggestion = many_generations_later()
    if len(suggestion.get_leaves()) == leaves:
        tree_state = suggestion
    return tree_state


In [273]:
log = []
for i, tree in enumerate(model.naive_monte_carlo(tree_generator=random_tree_with_memory, data=data.to_dict('list'), verbose=False)):
    if tree:
        print(i, tree.newick)
        tree.remove_internal_names()
        log.append(tree)
    if i > 10000:
        break


514 ((Kl:2.0,Ta:2.0)v••Δ••:3.0,(Sg:2.0,Nd:2.0)v•••••:3.0)v••:3.0
895 (Kl:7.0,((Ta:4.0,Sg:4.0)v•Δ•:2.0,Nd:6.0)v•:1)v:1
946 ((Kl:4.0,(Ta:2.0,Sg:2.0)v••Δ••:2.0)v••Δ:1,Nd:5.0)v••:3.0
1130 (Kl:7.0,(Ta:3.0,(Nd:1,Sg:1)v••••••:2.0)v••••:4.0)v:1
1239 ((Kl:4.0,Ta:4.0)vΔ••:3.0,(Sg:4.0,Nd:4.0)v•••:3.0)v:1
1382 (Kl:5.0,(Ta:2.0,(Nd:1,Sg:1)v••••••:1)v•••••:3.0)v••:3.0


1559 (Kl:7.0,(Ta:5.0,(Sg:3.0,Nd:3.0)v••••:2.0)v••:2.0)v:1
1621 (Kl:6.0,(Ta:5.0,(Nd:1,Sg:1)v••••••:4.0)v••:1)v•:2.0


4292 ((Kl:2.0,Ta:2.0)v•Δ•••:4.0,(Nd:1,Sg:1)v••••••:5.0)v•:2.0
4408 ((Kl:4.0,Ta:4.0)v••Δ:1,(Nd:1,Sg:1)v••••••:4.0)v••:3.0
4612 ((Kl:5.0,Ta:5.0)v•Δ:1,(Sg:4.0,Nd:4.0)v•••:2.0)v•:2.0
4623 ((Kl:5.0,(Ta:4.0,Sg:4.0)v•Δ•:1)v•Δ:1,Nd:6.0)v•:2.0
5224 (Kl:5.0,(Ta:3.0,(Nd:1,Sg:1)v••••••:2.0)v••••:2.0)v••:3.0
5227 (Kl:5.0,(Ta:3.0,(Nd:1,Sg:1):2.0):2.0):3.0


5321 ((Kl:4.0,(Ta:3.0,Sg:3.0)vΔ•••:1)vΔ••:3.0,Nd:7.0)v:1
5903 (Kl:6.0,((Ta:3.0,Sg:3.0)v•••Δ:1,Nd:4.0)v•••:2.0)v•:2.0
6105 (Kl:7.0,(Ta:3.0,(Sg:2.0,Nd:2.0)v•••••:1)v••••:4.0)v:1
6108 (Kl:7.0,(Ta:3.0,(Sg:2.0,Nd:2.0):1):4.0):1
6150 (Kl:7.0,(Ta:3.0,(Sg:2.0,Nd:2.0):1):4.0):1


6573 ((Kl:1,Ta:1)v•Δ••••:5.0,(Sg:2.0,Nd:2.0)v•••••:4.0)v•:2.0
6838 ((Kl:1,Ta:1)v••••Δ•:2.0,(Nd:1,Sg:1)v••••••:2.0)v••••:5.0


7770 ((Kl:2.0,Ta:2.0)v•Δ•••:4.0,(Nd:1,Sg:1)v••••••:5.0)v•:2.0
7874 (Kl:7.0,(Ta:3.0,(Nd:1,Sg:1)v••••••:2.0)v••••:4.0)v:1
8623 (Kl:5.0,(Ta:4.0,(Sg:2.0,Nd:2.0)v•••••:2.0)v•••:1)v••:3.0
8624 (Kl:5.0,(Ta:4.0,(Sg:2.0,Nd:2.0):2.0):1):3.0
8635 ((Kl:2.0,Ta:2.0)v•••Δ•:2.0,(Sg:2.0,Nd:2.0)v•••••:2.0)v•••:4.0


9541 (Kl:7.0,(Ta:3.0,(Nd:1,Sg:1)v••••••:2.0)v••••:4.0)v:1


In [286]:
for i, tree in enumerate(model.naive_monte_carlo(tree_generator=random_tree_with_memory, data=data.to_dict('list'), verbose=False)):
    if tree:
        tree.remove_internal_names()
        log.append(tree)
    if i > 100000:
        break

with open("fine_trees.nex", "w") as trees:
    newick.dump(log, trees)


In [287]:
from IPython.display import Image

Image(filename='densitrees.png')


<IPython.core.display.Image object>

In [288]:
raw_data

       Kalang    Nda'o Sunggama   Ta'e
child  tuˈmun     ˈana     anˈa   ˈana
moon      pak    ˈwuɹa    wuˈla  ˈʋula
flat    ˈrata  ˈn͡dena    deˈna  ˈrata

In [289]:
with open("true.tree") as original:
    print(newick.load(original)[0].ascii_art())
    

      ┌─Kl
──0───┤
      │     ┌─Ta
      └─0r──┤
            │     ┌─Nd
            └─0rr─┤
                  └─Sg
